# Centralized Solution of Multi-Regional Transmission Expansion Horizontal Investment Coordination Problem

This notebook contains code for a three zone test case. The case consists of an artificial merging of IEEE 14 nodes, IEEE 30 nodes, and a 5 node test system, respectively, for each zone. This notebook assumes there is a single entity (e.g. national planner) to solve the optimization problem centrally.
Results from this notebook serve a benchmark for the decentralized algorithms. 

In [1]:
# activate julia environment in a controlled way
julia_environment = "../central_control_PyJuMP/Julia_src/activate.jl"
include(julia_environment)

In [2]:
# activate required packages
using Pkg
using JuMP
using GLPK
using HiGHS
using Gurobi
#using CPLEX
#using Xpress
#using Cbc
#using Clp
using XLSX
using CSV
using DataFrames
using Random
#using PrettyPrint

# print package status
Pkg.status()

## Problem set up

Set up the example problem by reading the input data files from the `../Input_Data/` folder, then separate the data frames for each zone.

The input data includes: 

* `CandLine.csv` which contains information on shared _candidate lines_ between zones
* `CandLineInt.csv` which contains information on internal _candidate lines_ within each zone
* `SharedEline.csv` which contains information on existing _shared lines_ between zones
* `Tran.csv` which contains information on existing transmission lines within each zone (i.e. as defined in the cited IEEE test systems)
* `Gen.csv` which contains information on the generators as defined in the cited IEEE test systems
* `Load.csv` which contains information on the loads as defined in the cited IEEE test systems

After loading the input data, construct several DataFrames for each zone, i.e. for each set of lines, generators, loads, etc.


In [ ]:
# Read input files to set the problem
data_path = "../Input_Data/"

# note that network data are convertered xlsx data (not true csv files)
shared_cand = DataFrame(XLSX.readtable(data_path * "CandLine.csv", "Taul1", header=true)) # shared candidate lines
int_cand =  DataFrame(XLSX.readtable(data_path * "CandLineInt.csv", "Taul1", header=true)) # internal candidate lines
shared_ex =  DataFrame(XLSX.readtable(data_path * "SharedEline.csv", "Taul1", header=true)) # shared existing lines
int_ex = DataFrame(XLSX.readtable(data_path * "Tran.csv", "Taul1", header=true)) # internal existing lines
gen =  DataFrame(XLSX.readtable(data_path * "Gen.csv" , "Taul1", header=true)) # generator data
load =  DataFrame(XLSX.readtable(data_path * "Load.csv", "Taul1", header=true)) # load data

# read scenario and zone summary data. These are true CSV files, not XLSX
scen_prob = CSV.read(data_path * "Scenario_Probability.csv", DataFrame) # scenario probabilities
zone_summary = CSV.read(data_path * "Zone_Summary.csv", DataFrame) # region-number of nodes

In [ ]:
Random.seed!(123)
for i in 1:size(load,1)
    load.P_load1[i] = load.P_load1[i] * rand(24)
    load.P_load2[i] = load.P_load2[i] * rand(24)
    load.P_load3[i] = load.P_load3[i] * rand(24)
    load.P_load4[i] = load.P_load4[i] * rand(24)
end

In [ ]:
# Functionally define dataframes for each element (loads, generators, etc.) within each zone
# The left hand side is the singleton function that references the dataframe on the right hand side, e.g. l(1) is the load dataframe for zone 1


l(i,s) = load[load.zoneNum .== i, [1,2,s+2]] # load within zone i
g(i) = gen[gen.zoneNum .== i, :]   # generators within zone i
shared_c(i) = vcat(shared_cand[shared_cand.nodeZone1 .== i,:] , shared_cand[shared_cand.nodeZone2 .== i, :]) #shared candidate lines within zone i
int_c(i) = int_cand[int_cand.zoneNum .== i, :]   # number of internal candidate lines within zone i
shared_e(i) = vcat(shared_ex[shared_ex.nodeZone1 .== i,:] , shared_ex[shared_ex.nodeZone2 .== i, :]) #shared existing lines within zone i
int_e(i) =int_ex[int_ex.zoneNum .== i, :]       # internal existing lines within zone i
MC(i) = (g(i).C2 .* (g(i).PgMax .^ 2) .+ g(i).C1 .* g(i).PgMax .- g(i).C2 .*(g(i).PgMin .^ 2) .- g(i).C1.* g(i).PgMin) ./ (g(i).PgMax .- g(i).PgMin) #Marginal cost of generators within zone i
bin_c(i) = (shared_cand.nodeZone1 .== i) + (shared_cand.nodeZone2 .== i) # A binary vector through which we can check if the shared candidate lines belong to zone i
bin_e(i) = (shared_ex.nodeZone1 .== i) + (shared_ex.nodeZone2 .== i) # A binary vector through which we can check if the shared existing lines belong to zone i
scen_weight(s) = scen_prob.scen_weight[scen_prob.scenario .== s,:]

## Build the Optimization Problem in JuMP

Initialize the JuMP model object by calling `Model(ENGING.Optimizer)` where `ENGINE` is your optimization engine of choice, e.g. Gurobi, CPLEX, Xpress, HiGHS, etc. The optimization engine must have a solver for mixed integer linear programing (MIP). 

After initializing the model, we will add the variables, constraints, and objective function. 
Decision variables for the model (and their variable type) include:
> * Generator dispatch (float)
> * Decision on candidate shared lines (binary)
> * Decision on candidate internal lines (binary)
> * Power flow (float) and phase angle (float) on candidate shared lines
> * Power flow (float) and phase angle (float) on candidate internal lines
> * Power flow (float) and phase angle (float) on existing shared lines
> * Power flow (float) and phase angle (float) on existing internal lines

The objective function is the `total_cost` to build new lines and dispatch generation. The cost is defined as:
> \[EQUATION GOES HERE\]

The constraints include: 
> \[EQUATIONS GO HERE\]

In [ ]:
# Initialize the JuMP model object and set the optimization engine
# solvers = [Gurobi.Optimizer, Clp.Optimzer, Cbc.Optimizer, HiGHS.Optimizer, Ipopt.Optimizer, CPLEX.Optimizer, Xpress.Optimizer]

Mod = Model(Gurobi.Optimizer)

In [ ]:

zone_numbeddr = size(zone_summary,1)

In [ ]:
Tmax = 24

In [ ]:
# Add the decision variables to the model

# Decision variables for the power generation at each node
@variable(Mod,0 <= gen_var_1[1:14,1:nrow(scen_prob),1:Tmax]) # generation at node 1
@variable(Mod,0 <= gen_var_2[1:30,1:nrow(scen_prob),1:Tmax]) # generation at node 2
@variable(Mod,0 <= gen_var_3[1:5,1:nrow(scen_prob),1:Tmax]) # generation at node 3

# Decision variables for the candidate lines build/no build decision
@variable(Mod,shared_line_decision_var[1:nrow(shared_cand)], Bin) #Decision variable for shared candidate lines
@variable(Mod,int_line_decision_var_1[1:nrow(int_c(1))], Bin) #Decision variable for internal candidate lines of zone 1 (Binary)
@variable(Mod,int_line_decision_var_2[1:nrow(int_c(2))], Bin) #Decision variable for internal candidate lines of zone 2 (Binary)
@variable(Mod,int_line_decision_var_3[1:nrow(int_c(3))], Bin) #Decision variable for internal candidate lines of zone 3 (Binary)

# Decision variables for the power flow on the candidate lines
@variable(Mod, shared_cand_flow[1:nrow(shared_cand),1:nrow(scen_prob),1:Tmax])  #Power flowing on shared candidate lines 
@variable(Mod, int_cand_flow_1[1:nrow(int_c(1)),1:nrow(scen_prob),1:Tmax]) #Power flowing on internal candidate lines of zone 1 
@variable(Mod, int_cand_flow_2[1:nrow(int_c(2)),1:nrow(scen_prob),1:Tmax]) #Power flowing on internal candidate lines of zone 2
@variable(Mod, int_cand_flow_3[1:nrow(int_c(3)),1:nrow(scen_prob),1:Tmax]) #Power flowing on internal candidate lines of zone 3

# Decision variables for the phase angle of the candidate lines
@variable(Mod, 0 <= node_voltage_phase_angle[i=1:nrow(zone_summary),j=1:zone_summary.Nodes_Total[i], 1:nrow(scen_prob),1:Tmax]<= 2*pi)

# Decision variables for the power flow on the existing lines
@variable(Mod, shared_ex_flow[1:nrow(shared_ex),1:nrow(scen_prob),1:Tmax])  #Power flowing on shared existing shared lines

@variable(Mod, int_ex_flow_1[1:nrow(int_e(1)),1:nrow(scen_prob),1:Tmax])  #Power flowing on internal existing lines of zone 1 
@variable(Mod, int_ex_flow_2[1:nrow(int_e(2)),1:nrow(scen_prob),1:Tmax])  #Power flowing on internal existing lines of zone 2
@variable(Mod, int_ex_flow_3[1:nrow(int_e(3)),1:nrow(scen_prob),1:Tmax])  #Power flowing on internal existing lines of zone 3


In [ ]:
# Build the total_cost objective function one element and one zone at a time

# Initialize total_cost function
@expression(Mod,total_cost,0)

In [ ]:
# Define the scenario weighted gen cost for zone 1
@expression(Mod, gen_costNS1[n=1:14,s=1:nrow(scen_prob),t=1:Tmax], (gen_var_1[n,s,t] .* sum((g(1).gNodeID .== n) .* MC(1))))
@expression(Mod, gen_cost_prob_weightedS1[s=1:nrow(scen_prob),t=1:Tmax], (scen_weight(s)[1]).*sum(gen_costNS1[n,s,t] for n in 1:14))

# Add the expectation of gen cost for zone 1 to the total_cost function
@expression(Mod, gen_cost_expected1, sum(sum(gen_cost_prob_weightedS1[s,t] for s in 1:nrow(scen_prob)) for t in 1:Tmax))
Mod[:total_cost] += gen_cost_expected1


In [ ]:
# Define the scenario weighted gen cost for zone 2
@expression(Mod, gen_costNS2[n=1:30,s=1:nrow(scen_prob),t=1:Tmax], (gen_var_2[n,s,t] .* sum((g(2).gNodeID .== n) .* MC(2))))
@expression(Mod, gen_cost_prob_weightedS2[s=1:nrow(scen_prob),t=1:Tmax], (scen_weight(s)[1]).*sum(gen_costNS2[n,s,t] for n in 1:30))

# Add the expectation of gen cost for zone 2 to the total_cost function
@expression(Mod, gen_cost_expected2, sum(sum(gen_cost_prob_weightedS2[s,t] for s in 1:nrow(scen_prob)) for t in 1:Tmax))
Mod[:total_cost]+=gen_cost_expected2

In [ ]:
# Define the scenario weighted gen cost for zone 3
@expression(Mod, gen_costNS3[n=1:5,s=1:nrow(scen_prob),t=1:Tmax], (gen_var_3[n,s,t] .* sum((g(3).gNodeID .== n) .* MC(3))))
@expression(Mod, gen_cost_prob_weightedS3[s=1:nrow(scen_prob),t=1:Tmax], (scen_weight(s)[1]).*sum(gen_costNS3[n,s,t] for n in 1:5))

# Add the expectation of gen cost for zone 3 to the total_cost function
@expression(Mod, gen_cost_expected3, sum(sum(gen_cost_prob_weightedS3[s,t] for s in 1:nrow(scen_prob)) for t in 1:Tmax))
Mod[:total_cost]+=gen_cost_expected3

In [ ]:
# Add  the investment cost terms to the total_cost function
@expression(Mod, investment_cost , (sum(shared_line_decision_var[c] .* shared_cand.costPerCap[c] .* shared_cand.interestRate[c] 
            .*((1 + shared_cand.interestRate[c]) .^ shared_cand.lifeTime[c]) ./ (((1 + shared_cand.interestRate[c]) .^ shared_cand.lifeTime[c])-1) for c in 1:nrow(shared_cand))
            .+ sum(int_line_decision_var_1[c] .* int_c(1).costPerCap[c] .* int_c(1).interestRate[c] 
            .*((1 + int_c(1).interestRate[c]) .^ int_c(1).lifeTime[c]) ./ (((1 + int_c(1).interestRate[c]) .^ int_c(1).lifeTime[c])-1) for c in 1:nrow(int_c(1)))
            .+ sum(int_line_decision_var_2[c] .* int_c(2).costPerCap[c] .* int_c(2).interestRate[c] 
            .*((1 + int_c(2).interestRate[c]) .^ int_c(2).lifeTime[c]) ./ (((1 + int_c(2).interestRate[c]) .^ int_c(2).lifeTime[c])-1) for c in 1:nrow(int_c(2)))
            .+ sum(int_line_decision_var_3[c] .* int_c(3).costPerCap[c] .* int_c(3).interestRate[c] 
            .*((1 + int_c(3).interestRate[c]) .^ int_c(3).lifeTime[c]) ./ (((1 + int_c(3).interestRate[c]) .^ int_c(3).lifeTime[c])-1) for c in 1:nrow(int_c(3)))))
Mod[:total_cost]+=investment_cost

In [ ]:
# Define constraints for each zone

# Define constraints for Zone 1
# This includes nodal power balance constraints and upper and lower bound of connecting generators
for n in 1:14
    for t in 1:Tmax
        for s in 1:nrow(scen_prob)
            # Power balance constraint for each node
            @constraint(Mod, sum(g(1).gNodeID .== n) .* gen_var_1[n,s,t] .+ sum(l(1,s)[:,3].* (l(1,s).lNodeID .== n))[t]./100 .==
            sum((shared_cand.tNodeID1 .== n) .*bin_c(1) .* shared_cand_flow[:,s,t]) .- sum((shared_cand.tNodeID2 .== n) .* bin_c(1) .* shared_cand_flow[:,s,t]) .+
            sum((shared_ex.tNodeID1 .== n) .* bin_e(1) .* shared_ex_flow[:,s,t]) .- sum((shared_ex.tNodeID2 .== n) .* bin_e(1) .* shared_ex_flow[:,s,t]) .+
            sum((int_c(1).tNodeID1 .== n) .* int_cand_flow_1[:,s,t]) .- sum((int_c(1).tNodeID2 .== n) .* int_cand_flow_1[:,s,t]) .+
            sum((int_e(1).tNodeID1 .== n) .* int_ex_flow_1[:,s,t]) .- sum((int_e(1).tNodeID2 .== n) .* int_ex_flow_1[:,s,t]))
            #Lower limit for generation of each node
            @constraint(Mod, sum(g(1).gNodeID .== n) .* gen_var_1[n,s,t] .<= sum((g(1).gNodeID .== n) .* g(1).PgMax)./100)
            #Upper limit for generation of each node
            @constraint(Mod, sum((g(1).gNodeID .== n) .* g(1).PgMin)./100 .<= sum(g(1).gNodeID .== n) .* gen_var_1[n,s,t])
        end
    end
end

In [ ]:
# Define constraints for Zone 1
# This includes nodal power balance constraints and upper and lower bound of connecting generators

for n in 1:30
    for t in 1:Tmax
        for s in 1:nrow(scen_prob)
            # Power balance constraint for each node
            @constraint(Mod, sum(g(2).gNodeID .== n) .* gen_var_2[n,s,t] .+ sum(l(2,s)[:,3] .* (l(2,s).lNodeID .== n))[t]./100 .==
            sum((shared_cand.tNodeID1 .== n) .*bin_c(2) .* shared_cand_flow[:,s,t]) .- sum((shared_cand.tNodeID2 .== n) .* bin_c(2) .* shared_cand_flow[:,s,t]) .+
            sum((shared_ex.tNodeID1 .== n) .* bin_e(2) .* shared_ex_flow[:,s,t]) .- sum((shared_ex.tNodeID2 .== n) .* bin_e(2) .* shared_ex_flow[:,s,t]) .+
            sum((int_c(2).tNodeID1 .== n) .* int_cand_flow_2[:,s,t]) .- sum((int_c(2).tNodeID2 .== n) .* int_cand_flow_2[:,s,t]) .+
            sum((int_e(2).tNodeID1 .== n) .* int_ex_flow_2[:,s,t]) .- sum((int_e(2).tNodeID2 .== n) .* int_ex_flow_2[:,s,t]))
            #Lower limit for generation of each node
            @constraint(Mod, sum(g(2).gNodeID .== n) .* gen_var_2[n,s,t] .<= sum((g(2).gNodeID .== n) .* g(2).PgMax)./100)
            #Upper limit for generation of each node
            @constraint(Mod, sum((g(2).gNodeID .== n) .* g(2).PgMin)./100 .<= sum(g(2).gNodeID .== n) .* gen_var_2[n,s,t])
        end
    end
end

In [ ]:
# Define constraints for Zone 1
# This includes nodal power balance constraints and upper and lower bound of connecting generators

for n in 1:5
    for t in 1:Tmax
        for s in 1:nrow(scen_prob)
            # Power balance constraint for each node
            @constraint(Mod, sum(g(3).gNodeID .== n) .* gen_var_3[n,s,t] .+ sum(l(3,s)[:,3] .* (l(3,s).lNodeID .== n))[t]./100 .==
            sum((shared_cand.tNodeID1 .== n) .*bin_c(3) .* shared_cand_flow[:,s,t]) .- sum((shared_cand.tNodeID2 .== n) .* bin_c(3) .* shared_cand_flow[:,s,t]) .+
            sum((shared_ex.tNodeID1 .== n) .* bin_e(3) .* shared_ex_flow[:,s,t]) .- sum((shared_ex.tNodeID2 .== n) .* bin_e(3) .* shared_ex_flow[:,s,t]) .+
            sum((int_c(3).tNodeID1 .== n) .* int_cand_flow_3[:,s,t]) .- sum((int_c(3).tNodeID2 .== n) .* int_cand_flow_3[:,s,t]) .+
            sum((int_e(3).tNodeID1 .== n) .* int_ex_flow_3[:,s,t]) .- sum((int_e(3).tNodeID2 .== n) .* int_ex_flow_3[:,s,t]))
            #Lower limit for generation of each node
            @constraint(Mod, sum(g(3).gNodeID .== n) .* gen_var_3[n,s,t] .<= sum((g(3).gNodeID .== n) .* g(3).PgMax)./100)
            #Upper limit for generation of each node
            @constraint(Mod, sum((g(3).gNodeID .== n) .* g(3).PgMin)./100 .<= sum(g(3).gNodeID .== n) .* gen_var_3[n,s,t])
        end
    end
end

In [ ]:
# Deine constraints for the shared candidate lines
# The parameter big M is used to ensure equality constraints are satisfied when the candidate line is selected. 
# Choosing M is an art with some hueristic approaches. We want the smallest M that is still large enough to ensure the constraints are satisfied.
# For this problem, M should be related to the maximum power flow on the candidate line, assuming a 100 MW base power 
base_power = 100  # base power in MW
M = 500/base_power # M is 5 times the maximum power flow on the candidate line

for c in 1:nrow(shared_cand)
    for t in 1:Tmax
        for s in 1:nrow(scen_prob)
            @constraint(Mod,-M .* (1 .- shared_line_decision_var[c]) .<= shared_cand_flow[c,s,t] .- ((1 ./ shared_cand.reacT[c]) .* (node_voltage_phase_angle[shared_cand.nodeZone1[c],shared_cand.tNodeID1[c],s,t] .- node_voltage_phase_angle[shared_cand.nodeZone2[c],shared_cand.tNodeID2[c],s,t])))
            @constraint(Mod, shared_cand_flow[c,s,t] .- ((1 ./ shared_cand.reacT[c]) .* (node_voltage_phase_angle[shared_cand.nodeZone1[c],shared_cand.tNodeID1[c],s,t] .- node_voltage_phase_angle[shared_cand.nodeZone2[c],shared_cand.tNodeID2[c],s,t])) .<= M .* (1 .- shared_line_decision_var[c]))
            #limiting the upper bound of power flow flowing within candidate shared lines
            @constraint(Mod, shared_cand_flow[c,s,t] .<= shared_line_decision_var[c] .*shared_cand.ptMax[c]./100)
            #Limiting the lower bound of power flowing within the candidate shared lines
            @constraint(Mod, -(shared_cand.ptMax[c]./100) .* shared_line_decision_var[c] .<= shared_cand_flow[c,s,t])
        end
    end
end

In [ ]:
# Define constraints for existing shared lines
# These include power flow constraints and upper and lower bound of power flow on each line

for h in 1:nrow(shared_ex)
    for t in 1:Tmax
        for s in 1:nrow(scen_prob)
            @constraint(Mod, shared_ex_flow[h,s,t] .== (1 ./ shared_ex.reacT[h]) .* (node_voltage_phase_angle[shared_ex.nodeZone1[h],shared_ex.tNodeID1[h],s,t] .- node_voltage_phase_angle[shared_ex.nodeZone2[h],shared_ex.tNodeID2[h],s,t]))
            @constraint(Mod, shared_ex_flow[h,s,t] .<= shared_ex. ptMax[h]./100)
            @constraint(Mod, -shared_ex.ptMax[h]./100 .<= shared_ex_flow[h,s,t])
        end
    end
end

In [ ]:
# Define constraints for Zone 1 internal candidate lines
# These include power flow constraints and upper and lower bound of power flow on each line
# Note that we use the same parameter M as the shared candidate lines in this example. 
# In practice you may want to configure M differently for internal lines if they have different order of magnitude of power flow compared to shared lines

for c in 1:nrow(int_c(1))
    for t in 1:Tmax
        for s in 1:nrow(scen_prob)
            @constraint(Mod,-M .* (1 .- int_line_decision_var_1[c]) .<= int_cand_flow_1[c,s,t] .- ((1 ./ int_c(1).reacT[c]) .* (node_voltage_phase_angle[int_c(1).zoneNum[c],int_c(1).tNodeID1[c],s,t] .- node_voltage_phase_angle[int_c(1).zoneNum[c],int_c(1).tNodeID2[c],s,t])))
            @constraint(Mod, int_cand_flow_1[c,s,t] .- ((1 ./ int_c(1).reacT[c]) .* (node_voltage_phase_angle[int_c(1).zoneNum[c],int_c(1).tNodeID1[c],s,t] .- node_voltage_phase_angle[int_c(1).zoneNum[c],int_c(1).tNodeID2[c],s,t])) .<= M .* (1 .- int_line_decision_var_1[c]))
            @constraint(Mod, int_cand_flow_1[c,s,t] .<= int_line_decision_var_1[c] .* int_c(1).ptMax[c]./100)
            @constraint(Mod, -(int_c(1).ptMax[c]./100) .*int_line_decision_var_1[c] .<= int_cand_flow_1[c,s,t])
        end
    end
end

In [ ]:
# Define constraints for Zone 2 internal candidate lines
# These include power flow constraints and upper and lower bound of power flow on each line
# Note that we use the same parameter M as the shared candidate lines in this example. 
# In practice you may want to configure M differently for internal lines if they have different order of magnitude of power flow compared to shared lines

for c in 1:nrow(int_c(2))
    for t in 1:Tmax
        for s in 1:nrow(scen_prob)
            @constraint(Mod,-M .* (1 .- int_line_decision_var_2[c]) .<= int_cand_flow_2[c,s,t] .- ((1 ./ int_c(2).reacT[c]) .* (node_voltage_phase_angle[int_c(2).zoneNum[c],int_c(2).tNodeID1[c],s,t] .- node_voltage_phase_angle[int_c(2).zoneNum[c],int_c(2).tNodeID2[c],s,t])))
            @constraint(Mod, int_cand_flow_2[c,s,t] .- ((1 ./ int_c(2).reacT[c]) .* (node_voltage_phase_angle[int_c(2).zoneNum[c],int_c(2).tNodeID1[c],s,t] .- node_voltage_phase_angle[int_c(2).zoneNum[c],int_c(2).tNodeID2[c],s,t])) .<= M .* (1 .- int_line_decision_var_2[c]))
            @constraint(Mod, int_cand_flow_2[c,s,t] .<= int_line_decision_var_2[c] .* int_c(2).ptMax[c]./100)
            @constraint(Mod, -(int_c(2).ptMax[c]./100) .*int_line_decision_var_2[c] .<= int_cand_flow_2[c,s,t])
        end
    end
end

In [ ]:
# Define constraints for Zone 3 internal candidate lines
# These include power flow constraints and upper and lower bound of power flow on each line
# Note that we use the same parameter M as the shared candidate lines in this example. 
# In practice you may want to configure M differently for internal lines if they have different order of magnitude of power flow compared to shared lines

for c in 1:nrow(int_c(3))
    for t in 1:Tmax
        for s in 1:nrow(scen_prob)
            @constraint(Mod,-M .* (1 .- int_line_decision_var_3[c]) .<= int_cand_flow_3[c,s,t] .- ((1 ./ int_c(3).reacT[c]) .* (node_voltage_phase_angle[int_c(3).zoneNum[c],int_c(3).tNodeID1[c],s,t] .- node_voltage_phase_angle[int_c(3).zoneNum[c],int_c(3).tNodeID2[c],s,t])))
            @constraint(Mod, int_cand_flow_3[c,s,t] .- ((1 ./ int_c(3).reacT[c]) .* (node_voltage_phase_angle[int_c(3).zoneNum[c],int_c(3).tNodeID1[c],s,t] .- node_voltage_phase_angle[int_c(3).zoneNum[c],int_c(3).tNodeID2[c],s,t])) .<= M .* (1 .- int_line_decision_var_3[c]))
            @constraint(Mod, int_cand_flow_3[c,s,t] .<= int_line_decision_var_3[c] .* int_c(3).ptMax[c]./100)
            @constraint(Mod, -(int_c(3).ptMax[c]./100) .*int_line_decision_var_3[c] .<= int_cand_flow_3[c,s,t])
        end
    end
end

In [ ]:
# Define constraints for Zone 1 internal existing lines
# These include power flow constraints and upper and lower bound of power flow on each line

for h in 1:nrow(int_e(1))
    for t in 1:Tmax
        for s in 1:nrow(scen_prob)
            @constraint(Mod, int_ex_flow_1[h,s,t] .== (1 ./ int_e(1).reacT[h]) .* (node_voltage_phase_angle[int_e(1).zoneNum[h],int_e(1).tNodeID1[h],s,t] .- node_voltage_phase_angle[int_e(1).zoneNum[h],int_e(1).tNodeID2[h],s,t]))
            @constraint(Mod, int_ex_flow_1[h,s,t] .<= int_e(1).ptMax[h]./100)
            @constraint(Mod, -int_e(1).ptMax[h]./100 .<= int_ex_flow_1[h,s,t])
        end
    end
end

In [ ]:
# Define constraints for Zone 2 internal existing lines
# These include power flow constraints and upper and lower bound of power flow on each line

for h in 1:nrow(int_e(2))
    for t in 1:Tmax
        for s in 1:nrow(scen_prob)
            @constraint(Mod, int_ex_flow_2[h,s,t] .== (1 ./ int_e(2).reacT[h]) .* (node_voltage_phase_angle[int_e(2).zoneNum[h],int_e(2).tNodeID1[h],s,t] .- node_voltage_phase_angle[int_e(2).zoneNum[h],int_e(2).tNodeID2[h],s,t]))
            @constraint(Mod, int_ex_flow_2[h,s,t] .<= int_e(2).ptMax[h]./100)
            @constraint(Mod, -int_e(2).ptMax[h]./100 .<= int_ex_flow_2[h,s,t])
        end
    end
end

In [ ]:
# Define constraints for Zone 3 internal existing lines
# These include power flow constraints and upper and lower bound of power flow on each line

for h in 1:nrow(int_e(3))
    for t in 1:Tmax
        for s in 1:nrow(scen_prob)
            @constraint(Mod, int_ex_flow_3[h,s,t] .== (1 ./ int_e(3).reacT[h]) .* (node_voltage_phase_angle[int_e(3).zoneNum[h],int_e(3).tNodeID1[h],s,t] .- node_voltage_phase_angle[int_e(3).zoneNum[h],int_e(3).tNodeID2[h],s,t]))
            @constraint(Mod, int_ex_flow_3[h,s,t] .<= int_e(3).ptMax[h]./100)
            @constraint(Mod, -int_e(3).ptMax[h]./100 .<= int_ex_flow_3[h,s,t])
        end
    end
end

In [ ]:
# Collect the optimization problem and Solve

@objective(Mod, Min, Mod[:total_cost])
optimize!(Mod)

In [ ]:
# Collect the results after the model has completed

Centralized_results = Dict()   
if termination_status(Mod) == MOI.OPTIMAL
    println("Optimal solution found.")
    Centralized_results["shared_line_decision"] = value.(shared_line_decision_var)
    Centralized_results["shared_cand_power"] = value.(shared_cand_flow) .* base_power
    Centralized_results["node_voltage_phase_angle"] = value.(node_voltage_phase_angle) # this could be packaged up more nicely
    Centralized_results["shared_ex_power"] = value.(shared_ex_flow) .* base_power
    Centralized_results["int_line_decision_1"] = value.(int_line_decision_var_1)
    Centralized_results["int_line_decision_2"] = value.(int_line_decision_var_2)
    Centralized_results["int_line_decision_3"] = value.(int_line_decision_var_3)
    Centralized_results["int_line_flow_1"] = value.(int_cand_flow_1) .* base_power
    Centralized_results["int_line_flow_2"] = value.(int_cand_flow_2) .* base_power
    Centralized_results["int_line_flow_3"] = value.(int_cand_flow_3) .* base_power
    Centralized_results["generation_1"] = value.(gen_var_1) .* base_power
    Centralized_results["generation_2"] = value.(gen_var_2) .* base_power
    Centralized_results["generation_3"] = value.(gen_var_3) .* base_power
    Centralized_results["internal_existing_line_flow_1"] = value.(int_ex_flow_1) .* base_power
    Centralized_results["internal_existing_line_flow_2"] = value.(int_ex_flow_2) .* base_power
    Centralized_results["internal_existing_line_flow_3"] = value.(int_ex_flow_3) .* base_power

    Centralized_results["obj_value"] = objective_value(Mod)

    pprint(Centralized_results)
else
    println("Optimal solution not found.")
    value = JuMP.dual
end

